In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Reading the inputs

In [2]:
executions=pd.read_parquet("exectuions.parquet")

In [3]:
market_data=pd.read_parquet("marketdata.parquet")

In [4]:
ref_data=pd.read_parquet("refdata.parquet")

# Count the number of executions within the executions.parquet file, determine the unique number of [‘Venue’]s and the date of executions.

In [5]:
print(f"total number of excutions :- {executions.shape[0]}")

total number of excutions :- 4203


In [6]:
print(f"Unique number of venues :- {executions["Venue"].nunique()}")

Unique number of venues :- 6


In [7]:
print(f"Unique number of date of executions :- {pd.to_datetime(executions["TradeTime"]).dt.date.nunique()}")

Unique number of date of executions :- 1


# 2. Data Cleaning:
# a. Filter executions.paraquet for only CONTINUOUS_TRADING trades.
# b. Log output the # of executions.

In [8]:
countinuous_executions=executions[executions['Phase']=='CONTINUOUS_TRADING']

In [9]:
print(f"total number of excutions with phase countinuous trading :- {countinuous_executions.shape[0]}")

total number of excutions with phase countinuous trading :- 4103


In [10]:
countinuous_executions.head()

,ISIN,Currency,Venue,TradeTime,Price,Trade_id,Phase,Quantity
100,BE0003851681,EUR,XBRU,2022-09-02 07:39:39.072,91.80,100,CONTINUOUS_TRADING,11
101,BE0003851681,EUR,XBRU,2022-09-02 07:43:05.795,91.90,101,CONTINUOUS_TRADING,16
102,BE0003851681,EUR,XBRU,2022-09-02 07:47:55.688,91.85,102,CONTINUOUS_TRADING,22
103,BE0003851681,EUR,XBRU,2022-09-02 07:50:54.472,91.95,103,CONTINUOUS_TRADING,17
104,BE0003851681,EUR,XBRU,2022-09-02 07:54:06.487,92.05,104,CONTINUOUS_TRADING,23


# 3. Data Transformation:
# a. Add column [‘side’], if quantity is negative, side = 2, if quantity is positive side = 1.
# b. Complement the data with refdata.parquet
# i. Add the primary ticker [‘primary_ticker’]
# ii. Add the primary mic [‘primary_mic’]

In [11]:
countinuous_executions['side']=np.where(countinuous_executions['Quantity']>0,1,2)

In [12]:
countinuous_executions.sample(n=10, random_state=2)

,ISIN,Currency,Venue,TradeTime,Price,Trade_id,Phase,Quantity,side
2978,DE000BAY0017,EUR,XETA,2022-09-02 13:10:55.119,52.08,2978,CONTINUOUS_TRADING,42,1
966,DE000LEG1110,EUR,XETA,2022-09-02 15:07:58.038,75.02,966,CONTINUOUS_TRADING,24,1
2180,DE0006062144,EUR,XETA,2022-09-02 12:12:16.183,30.80,2180,CONTINUOUS_TRADING,62,1
308,CH0012005267,CHF,XSWX,2022-09-02 07:14:34.143,78.56,308,CONTINUOUS_TRADING,-12,2
2508,DE000BASF111,EUR,XETA,2022-09-02 12:05:44.204,42.29,2508,CONTINUOUS_TRADING,61,1
434,CH0363463438,CHF,XSWX,2022-09-02 08:51:26.822,14.36,434,CONTINUOUS_TRADING,-351,2
3543,DK0010181759,DKK,XCSE,2022-09-02 12:20:08.391,942.40,3543,CONTINUOUS_TRADING,20,1
783,DE000LEG1110,EUR,XETA,2022-09-02 07:30:21.988,73.76,783,CONTINUOUS_TRADING,24,1
3321,DK0010219153,DKK,XCSE,2022-09-02 12:20:31.072,1471.50,3321,CONTINUOUS_TRADING,9,1
2828,DE000BAY0017,EUR,XETA,2022-09-02 11:44:17.385,51.58,2828,CONTINUOUS_TRADING,12,1


In [13]:
%%time
# Joining with reference data
ref_updated_countinuous_executios=pd.merge(
    countinuous_executions[['ISIN', 'Currency', 'Venue', 'TradeTime', 'Price', 'Trade_id', 'Phase',
       'Quantity', 'side']],
    ref_data[['ISIN','Currency','primary_ticker','primary_mic','id']], how='left',left_on=['ISIN','Currency'],right_on=['ISIN','Currency'])

CPU times: user 7.67 ms, sys: 2.72 ms, total: 10.4 ms
Wall time: 9.47 ms


In [16]:
# casting the TradeTime column to datetime
ref_updated_countinuous_executios['TradeTime']=pd.to_datetime(ref_updated_countinuous_executios['TradeTime'])


# 4. Calculations:
# a. Best bid price and best ask (bbo) – bbo data in marketdata.parquet
# i. Find bbo price at execution, 1 second before execution and 1 second after execution
# from the marketdata.parquet file and add this data into the final output file –
# respective column table names [‘best_bid’,’best_ask’,
# ‘best_bid_min_1s’,’best_ask_min_1s’, ‘best_bid_1s’,’best_ask_1s’]

# b. Mid-Price – bbo data in marketdata.parquet
# i. Find the Mid-Price at execution, 1s before the execution and 1s after the execution
# – respective column table names [‘mid_price’, ‘mid_price_min_1s’ ‘mid_price_1s’]

# c. Calculate Slippage [‘slippage’] at execution price
# i. For SELL: (execution_price – best_bid) / (best_ask – best_bid)
# ii. For BUY : (best_ask – execution_price) / (best_ask – best_bid)

In [17]:
%%time
# calculating the best bid and best ask price across the market data for id 
market_data.rename(columns={'listing_id':'id'},inplace=True)
best_bid_ask_price=market_data.groupby('id').agg({'best_bid_price':[min],'best_ask_price':[max]}).reset_index()
best_bid_ask_price.columns=[f'{col[0]}_{col[1]}' for col in best_bid_ask_price.columns ]
best_bid_ask_price.rename(columns={'id_':'id','best_bid_price_min':'best_bid_price','best_ask_price_max':'best_ask_price'},inplace=True)

CPU times: user 47.9 ms, sys: 11.5 ms, total: 59.5 ms
Wall time: 60.3 ms


In [18]:
%%time
# calculating the best bid and best ask price for executions +1 
best_bid_ask_1=pd.merge_asof(
    ref_updated_countinuous_executios[['TradeTime','id','primary_mic']].sort_values('TradeTime'),
    market_data[['event_timestamp','best_bid_price','best_ask_price','id']].sort_values('event_timestamp'), 
    by='id',left_on='TradeTime',right_on='event_timestamp',direction='forward',tolerance=pd.Timedelta(seconds=1))

CPU times: user 197 ms, sys: 51.8 ms, total: 249 ms
Wall time: 250 ms


In [19]:
# renaming columns
best_bid_ask_1.rename(columns={'best_bid_price':'best_bid_1s','best_ask_price':'best_ask_1s','event_timestamp':'event_timestamp_1s'},inplace=True)

In [21]:
%%time
# calculating the best bid and best ask price for executions -1 
best_bid_ask_min_1=pd.merge_asof(
    ref_updated_countinuous_executios[['TradeTime','id','primary_mic']].sort_values('TradeTime'),
    market_data[['event_timestamp','best_bid_price','best_ask_price','primary_mic','id']].sort_values('event_timestamp'), 
    by='id',left_on='TradeTime',right_on='event_timestamp',direction='backward',tolerance=pd.Timedelta(seconds=1))

CPU times: user 243 ms, sys: 27.6 ms, total: 271 ms
Wall time: 271 ms


In [23]:
# renaming columns
best_bid_ask_min_1.rename(columns={'best_bid_price':'best_bid_min_1s','best_ask_price':'best_ask_min_1s','event_timestamp':'event_timestamp_min_1s'},inplace=True)


In [24]:
%%time
# joining to get best bid and ask price previously calculated
updated_executios=pd.merge(
    ref_updated_countinuous_executios[['ISIN', 'Currency', 'Venue', 'TradeTime', 'Price', 'Trade_id', 'Phase',
       'Quantity', 'side', 'primary_ticker', 'primary_mic', 'id']],
    best_bid_ask_price[['id','best_bid_price','best_ask_price']], how='left',left_on='id',right_on='id')

CPU times: user 6.03 ms, sys: 8.42 ms, total: 14.5 ms
Wall time: 12.9 ms


In [25]:
%%time
# joining to get best bid and ask 1 sec before exec price previously calculated
updated_executios_1s=pd.merge(
    updated_executios,
    best_bid_ask_1[['id','best_bid_1s','best_ask_1s','event_timestamp_1s']], how='left',left_on='id',right_on='id')
updated_executios_1s=updated_executios_1s.drop_duplicates()

CPU times: user 587 ms, sys: 155 ms, total: 743 ms
Wall time: 743 ms


In [26]:
%%time
# joining to get best bid and ask - 1 sec before exec price previously calculated
updated_executios_min_1s=pd.merge(
    updated_executios_1s,
    best_bid_ask_min_1[['id','best_bid_min_1s','best_ask_min_1s','event_timestamp_min_1s']], how='left',left_on='id',right_on='id')
updated_executios_min_1s=updated_executios_min_1s.drop_duplicates()

CPU times: user 4.29 s, sys: 1.13 s, total: 5.42 s
Wall time: 5.43 s


In [27]:
%%time
# calculating the mid price ,mid price _1 and mid_price -1

updated_executios_min_1s['mid_price']=(updated_executios_min_1s['best_bid_price']+updated_executios_min_1s['best_ask_price'])/2
updated_executios_min_1s['mid_price_1s']=(updated_executios_min_1s['best_bid_1s']+updated_executios_min_1s['best_ask_1s'])/2
updated_executios_min_1s['mid_price_min_1s']=(updated_executios_min_1s['best_bid_min_1s']+updated_executios_min_1s['best_ask_min_1s'])/2

CPU times: user 102 ms, sys: 99.1 ms, total: 201 ms
Wall time: 44.5 ms


In [28]:
%%time
# calculating the slippage based on side

updated_executios_min_1s['slippage']=np.where(
    updated_executios_min_1s['side']==1,
    (updated_executios_min_1s['best_ask_price']-updated_executios_min_1s['Price']) / (updated_executios_min_1s['best_ask_price']-updated_executios_min_1s['best_bid_price'])
    ,np.where(updated_executios_min_1s['side']==2,
             (updated_executios_min_1s['Price']-updated_executios_min_1s['best_bid_price']) / (updated_executios_min_1s['best_ask_price']-updated_executios_min_1s['best_bid_price']),
             np.nan))


CPU times: user 111 ms, sys: 103 ms, total: 213 ms
Wall time: 53.9 ms


In [29]:
final_df=updated_executios_min_1s
final_df.sample(n=10, random_state=25)

,ISIN,Currency,Venue,TradeTime,Price,Trade_id,Phase,Quantity,side,primary_ticker,...,best_bid_1s,best_ask_1s,event_timestamp_1s,best_bid_min_1s,best_ask_min_1s,event_timestamp_min_1s,mid_price,mid_price_1s,mid_price_min_1s,slippage
2259838,BE0003851681,EUR,XBRU,2022-09-02 12:51:31.488,93.20,190,CONTINUOUS_TRADING,23,1,AED,...,93.85,93.95,2022-09-02 13:35:44.275728311,93.45,93.55,2022-09-02 12:39:53.966468327,93.20,93.900,93.500,0.500000
2949082,BE0003851681,EUR,XBRU,2022-09-02 13:57:00.103,94.10,218,CONTINUOUS_TRADING,20,1,AED,...,92.85,93.00,2022-09-02 10:42:08.769535773,92.55,92.65,2022-09-02 10:02:57.271840963,93.20,92.925,92.600,0.250000
801402,BE0003851681,EUR,XBRU,2022-09-02 09:56:02.300,92.50,132,CONTINUOUS_TRADING,11,1,AED,...,92.10,92.20,2022-09-02 09:04:56.140493012,93.30,93.40,2022-09-02 13:09:53.976050735,93.20,92.150,93.350,0.694444
3945834,BE0003851681,EUR,XBRU,2022-09-02 15:27:55.073,94.25,258,CONTINUOUS_TRADING,1,1,AED,...,92.75,92.90,2022-09-02 10:38:55.342426582,94.15,94.25,2022-09-02 15:27:55.038490941,93.20,92.825,94.200,0.208333
1995692,BE0003851681,EUR,XBRU,2022-09-02 12:37:02.817,93.35,180,CONTINUOUS_TRADING,1,1,AED,...,92.15,92.25,2022-09-02 08:13:22.049845232,94.45,94.50,2022-09-02 14:43:00.671444127,93.20,92.200,94.475,0.458333
4666308,BE0003593044,EUR,XBRU,2022-09-02 14:06:00.560,100.60,1227,CONTINUOUS_TRADING,11,1,COFB,...,99.05,99.15,2022-09-02 07:54:09.150457159,100.70,100.80,2022-09-02 14:08:22.012049947,100.35,99.100,100.750,0.441860
709972,BE0003851681,EUR,XBRU,2022-09-02 09:49:54.106,92.45,128,CONTINUOUS_TRADING,46,1,AED,...,93.40,93.50,2022-09-02 12:35:48.192033959,92.20,92.30,2022-09-02 08:32:47.198346917,93.20,93.450,92.250,0.708333
4338410,CH0008742519,CHF,XSWX,2022-09-02 07:59:53.219,507.60,1009,CONTINUOUS_TRADING,11,1,SCMN,...,507.20,507.60,2022-09-02 07:40:21.346042000,507.00,507.20,2022-09-02 08:13:25.041972000,505.90,507.400,507.100,0.292683
1565090,BE0003851681,EUR,XBRU,2022-09-02 11:00:58.132,92.85,162,CONTINUOUS_TRADING,15,1,AED,...,93.80,93.85,2022-09-02 14:08:15.382577182,92.95,93.10,2022-09-02 10:19:50.912507140,93.20,93.825,93.025,0.597222
510776,BE0003851681,EUR,XBRU,2022-09-02 08:55:36.519,92.45,120,CONTINUOUS_TRADING,19,1,AED,...,93.35,93.40,2022-09-02 12:37:02.817086706,93.35,93.40,2022-09-02 12:37:02.624109896,93.20,93.375,93.375,0.708333
